In [158]:
import cudf
import numpy as np

In [ ]:
''' 
+++++ UTILITY ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code for the customer that owns this utility
    customer_code --> String --> Combined with user_code provides a unique identifier for the utility. Even this field is anonymized
    city --> String --> City where the utility is located
    address --> String --> (Anonymized) address of the utility location
'''

In [ ]:
''' 
+++++ CUSTOMER ATTRIBUTES +++++
    
    user_code --> String --> (Anonymized) code that identifies the customer
    nominative --> String --> (Anonymized) customer name
    sex --> String --> Sex of the customer. It could be ‘M’, ‘F’, ‘P’, with ‘P’ denoting that the customer is a commercial activity (VAT number)
    age --> Int --> Age of the customer, set to null for commercial activities (sex = ‘P’). Its value must be >= 18
'''

In [ ]:
''' 
+++++ INVOICE ATTRIBUTES +++++
    
    bill_id --> Int --> Invoice identifier
    F1_kWh --> Float --> kWh of electricity consumed in the F1 time slot
    F2_kWh --> Float --> kWh of electricity consumed in the F2 time slot
    F3_kWh --> Float --> kWh of electricity consumed in the F3 time slot 
    date --> Date --> Start date
    light_start_date --> Date --> Start date of electricity invoice
    light_end_date --> Date --> End date of electricity invoice
    tv --> Float --> Television fee to pay
    gas_amount --> Float --> Gas fee to pay
    gas_average_cost --> Float --> Average cost of gas
    light_average_cost --> Float --> Average cost of electricity
    emission_date --> Date --> Emission date
    supply_type --> String --> Supply type (‘light’, ‘gas’, ‘gas and light’)
    gas_start_date --> Date --> Start date of gas invoice
    gas_end_date --> Date --> End date of gas invoice
    extra_fees --> Float --> Extra fees to pay
    gas_consumption --> Float --> Consumed gas
    light_consumption --> Float --> Consumed electricity
    gas_offer --> Float --> Name of the subscribed gas plan (anonymized)
    light_offer_type --> String --> Kind of plan for the electricity (‘single zone’, ‘bizone’, etc.)
    light_offer --> String --> Name of the subscribed electricity plan (anonymized)
    total_amount --> Float --> gas_amount + light_amount + extra_fees
    howmuch_pay --> Float --> Overall amount to pay, computed as total_amount + tv
    light_amount --> Float --> Amount to pay for the electricity
    average_unit_light_cost --> Float --> Average cost for electricity
    average_light_bill_cost --> Float --> Average cost for the electricity invoice
    average_unit_gas_cost --> Float --> Average cost for gas
    average_gas_bill_cost --> Float --> Average cost for the gas invoice
    billing_frequency --> String --> Billing frequency (‘monthly’, ‘quarterly’, etc.)
    bill_type --> String --> Kind of invoice (False means a “standard bill”)
    gas_system_charges --> Float --> Extra gas fees
    light_system_charges --> Float --> Extra electricity fees
    gas_material_cost --> Float --> Costs for gas
    light_transport_cost --> Float --> Extra electricity fees
    gas_transport_cost --> Float --> Extra gas fees
    light_material_cost --> Float --> Costs for electricity
'''

In [ ]:
''' 
+++++ DATA INGESTION +++++
    • Read data from its source
    • Study how to deal with data that doesn’t fit in memory (!) --> Da chiede a Gagliardelli
    • Locate missing values
    • Locate outliers
    • Sort data
'''

In [152]:
from base import *
base = BaseDfBench()

In [155]:
# READ DATA FROM ITS SOURCE
# --- remove _sample one day

# We can use bill_id as the index for our dataset --> meglio di no
# base.load_dataset('/data/invoices_sample.csv', 'csv', index_col='bill_id')

base.load_dataset('/data/invoices_sample.csv', 'csv')
df = base.get_df()
columns = base.get_columns()
df

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
0,0,0.0,0.0,0.0,CIPRESSA,C23B8FC235DF5479FB28D81E827A59B819BB0748,0298E5E66B5653CDAACADD156261A2A916A56C7E,F,81,63D286C61D20D76E9C9317BEB8644D5EE45134E4,...,<NA>,5AE3987CBA311BCAADC6274D28A7FB14B13B53C3,<NA>,False,-0.06,<NA>,0.06,<NA>,-0.2,<NA>
1,1,81.0,62.0,76.0,ROCCAVIVARA,CBE6B021F41D589269FBC724C131CBFC9905D950,FDAE2B1E0934BB76255A727CC9F3F00AA39982D9,M,38,8053C7982DE8A18746A8F9F0D1DC4B0C33CAC0FA,...,<NA>,5F88D510670063B20E789E6453BC2F3FB0BB67F7,<NA>,False,<NA>,9.15,<NA>,5.68,<NA>,23.13
2,2,0.0,0.0,0.0,VIDRACCO,BFAEB566B3BB78B9ABD4F86DB3A78BFD8489013B,FEBE7F29FB854C477AFA7073C70F0E6EA81C6B00,M,53,4F053D54B9F1B6FE565498E7FA726501FD8FDD18,...,<NA>,D4B8928760E729127EB4EB532C86127325FBA468,<NA>,False,1.24,<NA>,-10.13,<NA>,-7.13,<NA>
3,3,0.0,0.0,0.0,MEZZOLOMBARDO,CBD30C382C72ED4A3D9DC9D11C13C155930E7C66,9220A3E83BDCE622F70A1456C7898687FD3EDAC8,M,50,115C7D34A79C66AFCDB4EB65262595B197B0F861,...,"1,08 €/smc",BD4DEF66D7EF2D3D683D9C381D2359AD24B41D9C,<NA>,False,-0.88,<NA>,23.68,<NA>,17.66,<NA>
4,4,0.0,0.0,0.0,PINETO,4D17C711CADED6C1E9BD459088B4B80F24575FE7,642C21354EF6017D13220517E2A25D4A0442D4C5,M,65,C7734B97A212004CEA25956D8FDFEF068CD2B8F6,...,"0,86 €/smc",C0622AFD22384A4B54CD1ACAF7D988B89853A5AC,<NA>,False,14.71,<NA>,141.57,<NA>,63.59,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,26.0,25.0,33.0,CINTANO,2D244E5D90065985546AEA4D52F54C9705B8038B,102CADC9FC8C745FE3B8C27F7267E07C9691728A,M,55,6D3FDD0A2EEEE1A2C1AC23E6C316048DD2CCE859,...,<NA>,53D29ACDEB11F45D4F305A4D00E5F03C1F2CB674,monthly,False,<NA>,13.08,<NA>,7.62,<NA>,15.65
1996,1996,71.0,61.0,88.0,TREZZONE,A01243282C24F561D6B54F1C1FCB45AE80F75B0A,9C3B5B4093FC99569E5BBA918A977681A46F52DF,M,48,52315237533E1A6504E3795EAC1D5586744C62FD,...,<NA>,2B8D02A2AD117EC401BCEF3C0651FEEBB6609D1C,monthly,False,<NA>,9.2,<NA>,8.76,<NA>,25.8
1997,1997,37.0,71.0,100.0,RIPATRANSONE,469AB252FF0A2C60B32D0337C933E3565E88C0C2,1B451B6AA312EFF93372D44EC6C24C0110A628A1,F,32,2777855F19667FBFEDEAEF181E7ABC8584262127,...,<NA>,A1BFA0E4833E27AB398F4420414E037D250F0A7E,monthly,False,<NA>,8.7,<NA>,8.66,<NA>,24.14
1998,1998,74.0,78.0,92.0,MONSELICE,9E24BB26322BD95D11E26DFB80AA1168C9661FDC,F6BADD2456E283A8DCA6F3FF640724072108BAEF,F,54,8DC8820FB50005ED9CC4E7C187B9703460941856,...,<NA>,DED078288677E4E8A15874405E79C26AC78747B0,monthly,False,<NA>,10.21,<NA>,8.96,<NA>,30.43


In [ ]:
#verica di user_code+customer_code = PK utility
df.groupby(by=['user_code', 'customer_code']).count()

#PK customer = user_code

#PK ivoice = bill_id


In [ ]:
# STUDY HOW TO DEAL WITH DATA THAT DON'T FIT IN MEMORY (!)
# Bella storia
!du -sh '/data/invoices_sample.csv'

In [ ]:
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
from dask.utils import parse_bytes
import cudf
import dask_cudf

In [ ]:
cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("15GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("10GB"),
)
client = Client(cluster)
client

In [ ]:
%%time
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="1 GiB")
print(ddf.npartitions)
type(ddf)

In [ ]:
ddf.tail()

In [ ]:
ddf.head()

In [ ]:
df = ddf.loc['0':'4']
df

In [ ]:
df['date']

In [123]:
# LOCATE MISSING VALUES
rows = df.shape[0]
#wait(ddf.shape[0])
print("Total rows", rows)

# These are the rows with missing values
for c in columns:
    if (not base.locate_null_values(c).empty):
        print(c, "\t Affected rows:", base.locate_null_values(c).shape[0])
    #else:
        #print(c, "\t Qui no missing values")

Total rows 2000
address 	 Affected rows: 4
nominative 	 Affected rows: 4
gas_amount 	 Affected rows: 1120
gas_average_cost 	 Affected rows: 1496
light_average_cost 	 Affected rows: 1122
gas_consumption 	 Affected rows: 1120
light_consumption 	 Affected rows: 744
light_amount 	 Affected rows: 744
average_unit_light_cost 	 Affected rows: 1115
average_light_bill_cost 	 Affected rows: 1122
average_unit_gas_cost 	 Affected rows: 1496
average_gas_bill_cost 	 Affected rows: 1494
billing_frequency 	 Affected rows: 1569
gas_system_charges 	 Affected rows: 1155
light_system_charges 	 Affected rows: 776
gas_material_cost 	 Affected rows: 1155
light_transport_cost 	 Affected rows: 775
gas_transport_cost 	 Affected rows: 1155
light_material_cost 	 Affected rows: 775


In [ ]:
#Visto che address e nominative hanno lo stesso numero di NA, vale la 
#pena controllare ed eventualmente eliminare le righe?

base.locate_null_values('nominative')

In [ ]:
gas = df[['average_gas_bill_cost','gas_consumption','gas_amount','gas_average_cost','gas_transport_cost', 'gas_system_charges', 'gas_material_cost']]
gas

In [ ]:
#search by pattern

import re, itertools
gas_list = base.get_df().columns.str.contains(re.compile('date')).tolist()
#print(gas_list)
#df.loc[:, itertools.compress(df.columns.tolist(), gas_list)].drop('gas_offer', axis=1)
df.loc[:, itertools.compress(df.columns.tolist(), gas_list)]

In [ ]:
# Le date potrebbero fornirci informazioni sulla billing_frequency, o viceversa, come lo stabiliamo?
# Cosa indica la colonna dell'emission_date??

df.loc[0, 'billing_frequency']

In [ ]:
# LOCATE OUTLIERS

In [ ]:
from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import dask_cudf
import time

cluster = LocalCUDACluster(
    CUDA_VISIBLE_DEVICES="0",
    rmm_pool_size=parse_bytes("150GB"), # This GPU has 16GB of memory
    device_memory_limit=parse_bytes("100GB"),
)
client = Client(cluster)

time.sleep(2)
print("cluster status ", cluster.status)
print("cluster information ", cluster)
print("client information ",client)
client

In [ ]:
ddf = dask_cudf.read_csv("/data/invoices.csv", chunksize="0.1 GiB")
print(ddf.npartitions)
ddf = ddf.quantile(q=0.5)
_ = wait(ddf)

In [ ]:
!nvidia-smi

In [ ]:
ddf.compute()

In [ ]:
def column_outliers(x:cudf.Series, k:float):
    q1 = x.quantile(q=0.25)
    q3 = x.quantile(q=0.75)
    I = q3 - q1
    return (x < q1 - k*I) or (x > q3 + k*I)

def df_outliers(df:cudf.DataFrame, k:float):
    q1 = df.quantile(q=0.25)
    q3 = df.quantile(q=0.75)
    I = q3 - q1
    return (x < q1 - k*I) or (x > q3 + k*I)

In [ ]:
col_outliers(df.iloc[:,1], 0)

In [ ]:
type = ddf.dtypes
num = type[type!='object'][type!='bool']
num.index

In [ ]:
num_df = ddf[num.index].quantile(q=0.2)
num_df = num_df.persist()
num_df.compute()

In [ ]:
# SORT DATA
# Idee: ordinare per data

In [ ]:
''' 
+++++ DATA VALIDATION +++++
    • Check data range
    • Check column uniqueness
    • Find data-mismatched data types
'''

In [ ]:
# CHECK DATA RANGE

In [166]:
# Controllo se chi ha P ha età nulla
df.loc[(df['sex']=='P') & (df['age']>=0), 'age'] = np.nan
df.loc[(df['sex']=='P') & (np.isnan(df['age']))]

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
5,5,0.0,0.0,0.0,VALGRANA,CF9E4E3F659E5A7A5C9975570E66C5985FF20486,B2D49CC3EA712D9A6F333F57EB9280601924EC37,P,NaN,D3BFE563AB14EA63F311B8A25B3AA8F786718CCD,...,"0,86 €/smc",9ACA67CB28293E711CCA5F9B4FB0B0373FB5B36C,<NA>,False,16.97,<NA>,167.54,<NA>,44.45,<NA>
23,23,0.0,0.0,0.0,PONTERANICA,A5BECF7E67B192B00C4B68BA490479C036BE324E,AACEECF6549DC6B1D7E74EE72C93C53B7FAA7821,P,NaN,9CE05516185F3276F00525216AB198FE14B5113F,...,<NA>,95F77EB44034D838442434D344852734EA5BDAC0,<NA>,False,6.75,<NA>,-19.23,<NA>,-122.49,<NA>
64,64,22.0,0.0,0.0,TREVENZUOLO,67C880B1F8360FFD61CA876277C27AE42CC1E3F3,A59EA7A4097DDCAA37A3FBE515C065188191456C,P,NaN,3C9F70DE0664CBFE7B11F547A2A6B63BA05888D2,...,<NA>,FE750C0FA1EDDAA3540E42F2CA79A721BD95B160,<NA>,False,<NA>,1.09,<NA>,0.2,<NA>,1.83
67,67,22.0,0.0,0.0,SALERANO SUL LAMBRO,EAE353697612C5290B53F13E23885B1CA90AA95F,A59EA7A4097DDCAA37A3FBE515C065188191456C,P,NaN,3C9F70DE0664CBFE7B11F547A2A6B63BA05888D2,...,<NA>,B1292FFCC462370CA20E6C57B2AFF3BBBE131B89,<NA>,False,<NA>,1.09,<NA>,0.2,<NA>,1.83
71,71,6513.0,5302.0,6610.0,SAN MARTINO IN PENSILIS,8A8AC87E60AABF6E2BC6C4425707B36DC9342769,E5E3636009814DCE7967BD47310860DB493AA915,P,NaN,9D25EFBF38408B492D5C613EA3113B87285872EA,...,<NA>,B2E4410FBDBDB81768C594A8D81CA38462ABC32F,<NA>,False,<NA>,750.58,<NA>,190.01,<NA>,1100.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974,1974,75.0,59.0,103.0,BOTRUGNO,6ACE234AF417B73B6B0A9F8E9DE0349FB4BED941,DE7E423B3CD8E06D1ED6D379547B6BE1482D0AC8,P,NaN,FF4B3953260A64357143F82FD2881C2FDE653DE5,...,<NA>,F9880177E7100F50F3A24B5E0727FEFE311AD3EA,monthly,False,<NA>,40.63,<NA>,29.88,<NA>,35.03
1976,1976,25.0,20.0,32.0,SALUGGIA,E0199D57164E64907230FE6234D28BE78273A07D,1B5044B4D2E942ECDE6E1F29274F63278B484ED2,P,NaN,E22BE1CBFB3B1C22762058B95867C8D13C1BF67E,...,<NA>,1BB0AAB581F05A9D4EDF1FFA0E7E8346D248C33B,monthly,False,<NA>,13.16,<NA>,9.68,<NA>,19.47
1978,1978,40.0,10.0,23.0,SAN MICHELE AL TAGLIAMENTO,3BC5B84C086D75ADD1E4158DA859AFC0DE786856,797EFCCE73BFFF7CC9D4707CA38FC7EA68DC4D3D,P,NaN,5707197A4C338B375C232D807A117879EE9CDD0B,...,<NA>,614D6695D3993C36C44BA4F33817A09C4EED3989,monthly,False,<NA>,12.96,<NA>,9.64,<NA>,20.26
1980,1980,128.0,70.0,113.0,MALVICINO,DC05160B563E3CD82EF4F2A2D45F82E9B48154C7,091E23C5253C4DEB06F086D8604499615B413C6C,P,NaN,40BF5FE9E0EF59A68C0BA0DF41EA2348066384F2,...,<NA>,DB010BAE2C7A146433B38CF023FC6DF54CEF4EAF,monthly,False,<NA>,24.81,<NA>,11.77,<NA>,39.51


In [167]:
# Age >= 18
# Age non ha valori nulli (già controllato in precedenza IN TEORIA)
#
df['age'].isna().sum()

211

In [168]:
# Che fare con i minorenni?
df[df['age'] < 18]

,bill_id,F1_kWh,F2_kWh,F3_kWh,city,address,nominative,sex,age,user_code,...,average_gas_bill_cost,customer_code,billing_frequency,bill_type,gas_system_charges,light_system_charges,gas_material_cost,light_transport_cost,gas_transport_cost,light_material_cost
125,125,0.0,0.0,0.0,FOGLIANO REDIPUGLIA,66B3056AA7A65BAFADFB2FC19B5926A89632E126,F456AD97543E5C49CD295E4E5119F4050CB55C4E,F,2.0,58229C3FAAB50724313B04B8124833D2B3D6B3D8,...,<NA>,A3256354A9D6F2C36BAE7FBF3EEF6A85EA546D9C,<NA>,False,2.25,<NA>,-5.3,<NA>,-6.79,<NA>
306,306,884.0,634.0,892.0,ACQUANEGRA CREMONESE,9821F0A77D4F66D3142AE1C8C75B982AD1F5BF33,EBC7B7022B5CF5DFF8091D7BCA9314F213109E33,M,0.0,C40161075F1291FF89A41AA0C2B23DB3320739B6,...,<NA>,D6D318A2659F9C032DBF9BA3504F6DE3BA1AD06C,<NA>,False,<NA>,43.84,<NA>,17.73,<NA>,56.84
516,516,0.0,0.0,0.0,SALUGGIA,1A680AFD23C685FDC232BAFEFD79DBBBA3B74AE4,9B0E82AD1089A2D249F65E9EA7926218BB4BBE92,M,2.0,FF8BBDB3829B19DF0777B04D29EDF24E4D144326,...,"2,55 €/smc",6AA145DBE10586E62C216B9949C76459D4CEADD5,<NA>,False,-7.02,<NA>,20.86,<NA>,21.06,<NA>
698,698,0.0,0.0,0.0,ACQUANEGRA CREMONESE,D07AECCF706D03A06C6A7E12B8FA2E6035BC0335,6B90E90AAE8E9ACB46617B3196DAA8C31FD1D482,M,1.0,A1F5BDC32C90AAFF53B4B0364CAACAA3A76BF8E2,...,"1,16 €/smc",03832CCEEFA81FFBDAE9D92307D38442E8FEA407,<NA>,False,-1.07,<NA>,4.69,<NA>,3.73,<NA>
765,765,0.0,0.0,0.0,TORREBELVICINO,2BAD20EAF519E0D29C59CB929DA846518E10FAC7,642E4428F66BF4C1F9A4CECE3035F764463E7F2E,M,0.0,076C995A8D8E60C09AA3CD59206E88E7CB0063FB,...,"1,03 €/smc",55F14A2CF62989C287674EF8BD294E5F49568B3F,<NA>,False,1.47,<NA>,14.18,<NA>,7.57,<NA>
944,944,0.0,0.0,0.0,VEROLANUOVA,B5EE69E2777322478C1EECFED78677183048F6A7,069CCF218F8DDE1FAF2563C043B39E813534A94F,M,0.0,775BF75982DE7813A74537DD4E87BC8107DE31F5,...,<NA>,2E84D4546FFBCFB210D6EBDFD0F129943BC04AB3,<NA>,False,-0.52,<NA>,2.31,<NA>,1.07,<NA>
1064,1064,80.0,70.0,92.0,MONSELICE,817E0B870446A0B6557BE8067250CB5E929E1F47,42BED2A1F25834CF051DC58E61AE6D7909FDAD3C,M,0.0,0B14B0CEABB9A940AF48259D3F905663E8D98E02,...,<NA>,A0B2A3847A6554733F66ED5456F3A2822873CD1C,<NA>,False,<NA>,10.1,<NA>,4.51,<NA>,65.06
1067,1067,34.0,22.0,28.0,ARDEA,5BE32770075C58C6655F9AEF1A299ACF7CE72C5D,F8700002ABCC189F677E9BDD69C6C9750AD4CE4D,M,1.0,89BE766E36C1F35E4ADE1B6C62625FD92289940B,...,<NA>,BD259CF2510193E47E2084D1799FF312676FCA2D,<NA>,False,<NA>,3.51,<NA>,2.97,<NA>,7.35
1422,1422,-191.0,-18.0,-50.0,ACQUANEGRA CREMONESE,8BD58EF949AE93644264E1E2594DDACA386FA6A8,06DBFCF2A1EA7ACEB700DFD153A3DB57AE3D1B43,M,0.0,AC4DCDF5326997CE81185EF2335F4B4C8E74CC26,...,<NA>,282DB4270BF9DF602A3CCB356E5DD56AC87BC3DE,<NA>,False,<NA>,-0.8,<NA>,0.76,<NA>,-0.01
1429,1429,-29.0,-33.0,-39.0,SAN MARTINO D'AGRI,B38973211A1B0B0C76D0212F9DD2B294B3E05A0C,5BF62FD4508315E6DA7BF25EABBD0E72622B3378,M,0.0,EED3E7C644492705E647CCD3C8E0D596010461A4,...,<NA>,9CD065E9EEDD74BA4F728B40350659FCE3FF6323,<NA>,False,<NA>,1.43,<NA>,5.31,<NA>,1.05


In [206]:
idx = df['user_code'].value_counts()
idx

3C9F70DE0664CBFE7B11F547A2A6B63BA05888D2    4
17CB3BA9FE2A2B515718DA60309B9B37A37D0442    3
00106F4E6FEEFB8C81BADE0E7E985D85DFA3B9D9    2
00C7D7B84EF973F1D48AFAD0DDE706A611F0D60F    2
05ED97BCA5EB36FB909C6F37E3D62FB4AC897A1B    2
                                           ..
FFBCAFB0F9ABF49EE7CFE9169E1F6C09AE55711B    1
FFCCFA289099A7FF44726330B8B2C8730411C950    1
FFD9607191459984619642FFDE109415FBD135D4    1
FFDFE2EDE8A46EF89F4E670028002763FC719E22    1
FFE754BB2D984795616DAF79F744A209E4770B12    1
Name: user_code, Length: 1975, dtype: int32

In [ ]:
# CHECK COLUMN UNIQUENESS

In [ ]:
# The story changes here, we got these pairs as candidates as duplicate columns 
# (those which have same values for each row.)

# ATTENZIONE!! Probabile che le date del gas siano invertite, trovare un modo per confermarlo

In [ ]:
# No duplicates found on the rows, what about columns?
base.get_duplicate_columns()

In [ ]:
#FUFFA

compariser = cudf.Series(np.random.randn(2000))
print(df['gas_average_cost'])
print(df['average_unit_gas_cost'])
print(compariser.shape)
compariser = (df['gas_average_cost'] == df['average_unit_gas_cost']).astype(int)
print(compariser.sum())

In [ ]:
# FIND DATA MISMATCH

In [ ]:
# per le date: https://stackoverflow.com/questions/64615617/convert-date-strings-with-italian-month-names-to-y-m-d